<a href="https://colab.research.google.com/github/orbenishay/pandas-project/blob/master/Bicycle_6_Hagai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import pandas as pd
import matplotlib.pyplot as plt # visualization, EDA
import numpy as np  # Mathematical calculations
%matplotlib inline


In [82]:
if 'google.colab' in sys.modules:
    from google.colab import files
    uploaded = files.upload()

Saving trip.csv to trip (2).csv


In [112]:
# Hagai's Analysis:
# -----------------------------------------------------------------------------------------
# Read csv's
df_trip = pd.read_csv("trip (2).csv",
                   index_col='id', parse_dates=True, dayfirst=True)
df_trip['start_date'] = pd.to_datetime(df_trip['start_date'], errors='coerce', dayfirst=True)
df_trip['end_date'] = pd.to_datetime(df_trip['end_date'], errors='coerce', dayfirst=True)
df_trip['date_day']=df_trip['start_date'].dt.date
df_trip.head(20)
df_trip['duration']=df_trip['end_date']-df_trip['start_date']
df_trip.describe()
df_trip.sort_values('dur_adj', ascending=False).head(8)

# df_trip['duration_minutes'] = pd.DatetimeIndex(df_trip['duration']).minutes
df_trip['duration_minutes']=pd.to_numeric(df_trip['duration'])/60000000000
df_trip.hist(column='duration_minutes')



df_station = pd.read_csv("station.csv", index_col = 'id')
df_station.head()

df_bikes = pd.read_csv("bikes.csv", index_col = 'bike_id')
df_bikes.head()

df_weather = pd.read_csv("weather.csv", parse_dates=['date'])
df_weather['date_ec'] = pd.to_datetime(df_weather['date'], errors='coerce')
df_weather['date_day']=df_weather['Date_ec'].dt.date
df_weather.head()

# -----------------------------------------------------------------------------------------
# Merges
df_trip_weather = df_trip.merge(df_weather, how='left', left_on = ['zip_code', 'date_day'], right_on=['zip_code','date_day'])
# df_trip_weather.head()
df_trip_weather_station = df_trip_weather.merge(df_station, how='left', left_on = 'start_station_id', right_on='id')
# df_trip_weather_station.head()
df_all = df_trip_weather_station.merge(df_bikes, how='left', left_on = 'bike_id', right_on='bike_id')
df_all.head()

# ----------------------------------------------------------------------------------------
# Year/Month aggregation

df_all['year'] = pd.DatetimeIndex(df_all['start_date']).year
df_all['month'] = pd.DatetimeIndex(df_all['start_date']).month
df_all['hour'] = pd.DatetimeIndex(df_all['start_date']).hour
df_all['weekday'] = pd.DatetimeIndex(df_all['start_date']).weekday
df_group=df_all.groupby(['month','year']).agg({'start_date':'count'}).unstack(-1)
df_group.plot(kind='bar', stacked=False, figsize=(12, 6), by='year')
df_group.head(20)

# Duration
df_all['start']=pd.to_datetime(df_all['start_date'])
df_all['end']=pd.to_datetime(df_all['end_date'])
df_all['duration']=df_all['end']-df_all['start']
df_all['duration_minutes']=pd.to_numeric(df_all['duration'])/60000000000



df_all['duration']=df_all['end_date']-df_all['start_date']


df_all.boxplot(column='duration_minutes', by='city')
df_all['duration'].describe()

df_all.groupby(['city']).agg({'duration_minutes':'mean'}).head()


.hist(column='duration', by='city', bins=50)
df['time_delta'].astype('timedelta64[s]').plot.hist()

# ------------------------------------------------------------------------------------------
# rents distribution by city
df_station = pd.read_csv("station.csv", index_col = 'id')
df_trip_station = df_trip.merge(df_station, how='left', left_on = 'start_station_id', right_on='id')
# df_trip_station.head()
df_group_city_station=df_trip_station.groupby(['city','station_name']).agg({'start_date':'count'})
df_group_city_station.boxplot(column='start_date', by='city')
df_group_city_station.groupby('city').describe().unstack(0)
df_group_city_station.head()

# ------------------------------------------------------------------------------------------
# count rent by hour & weekday

# ------------------------------------------------------------------------------------------
# duration distribution






/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


KeyError: ignored